In [5]:
!pip install pandas scikit-learn datasets

In [2]:
import pandas as pd
import json
import re
from sklearn.model_selection import train_test_split

In [6]:
from datasets import load_dataset

ds = load_dataset("neelblabla/enron_labeled_emails_with_subjects-llama2-7b_finetuning")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/137 [00:00<?, ?B/s]

prompts_train.csv:   0%|          | 0.00/6.77M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [20]:
df = pd.DataFrame(ds['train'])

In [37]:
def extract_prompt_and_completion(text):
    """Extract prompt (<<SYS>> to [/INST]) and completion ([/INST] to <s>) using regex."""
    prompt_pattern = r'<<SYS>>(.*?)\[/INST\]'
    completion_pattern = r'\[/INST\](.*?)<s>'

    prompt_match = re.search(prompt_pattern, text, re.DOTALL)
    completion_match = re.search(completion_pattern, text, re.DOTALL)

    prompt = prompt_match.group(1).strip() if prompt_match else None
    completion = completion_match.group(1).strip() if completion_match else None

    return prompt, completion

In [43]:
df[['prompt', 'completion']] = df['prompts'].apply(lambda x: pd.Series(extract_prompt_and_completion(x)))
df = df.dropna(subset=['prompt', 'completion'])

In [38]:
p, c = extract_prompt_and_completion(df['prompts'][0])

In [45]:
train_df, temp_df = train_test_split(df[['prompt', 'completion']], test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [46]:
def save_jsonl(df, file_path):
    """Save dataframe as JSONL file."""
    with open(file_path, 'w') as f:
        for _, row in df.iterrows():
            json.dump(row.to_dict(), f)
            f.write('\n')

In [47]:
save_jsonl(train_df, '/content/train.jsonl')
save_jsonl(valid_df, '/content/valid.jsonl')
save_jsonl(test_df, '/content/test.jsonl')

print("✅ Datasets saved as JSONL files:")
print("- /content/train.jsonl")
print("- /content/valid.jsonl")
print("- /content/test.jsonl")


✅ Datasets saved as JSONL files:
- /content/train.jsonl
- /content/valid.jsonl
- /content/test.jsonl


In [48]:
!zip -r mail-category.zip mail-category/

  adding: mail-category/ (stored 0%)
  adding: mail-category/test.jsonl (deflated 66%)
  adding: mail-category/valid.jsonl (deflated 71%)
  adding: mail-category/train.jsonl (deflated 68%)


In [49]:
from google.colab import files
files.download('mail-category.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>